## Optimal strategy for two players luck/skill games - a new approach?

Theoretical background: have a look at https://bg1992.home.blog/2020/08/31/optimal-strategy.

Let us consider the following two players game, including both luck and skill:

First player starts with a number $0$, second player with a number $2^{2N}-1$. Players alternate moves which are subject to the rules:

1. Roll two dices labelled from $1$ to $N$.

2. Perform one of the operations on your current number $i$: $i \ xor \ 2^{x-1}$, $i \ xor \ 2^{y-1}$,$i \ xor \ 2^{x+y-1}$, where $xor$ is a bitwise operation: https://en.wikipedia.org/wiki/Bitwise_operation. Chosen number of those three above is your current number, on condition that it is not opponent's current number.

A player wins when his number is equal to the opponent's initial number. Therefore, first player wins, if after some moves manages to get the number $2^{2N}-1$.

What is the optimal strategy for both players? Let us perform the analysis for $N=4$.

In [16]:
N = 4
curr_scores = {}
tolerance = pow(10,-8)

In [17]:
#Generating all states, marking the terminals.
from random import uniform
for i in range(pow(2,2*N)):
    for j in range(pow(2,2*N)):
        if i != j:
            if i == pow(2,2*N)-1 and j == 0: score = None
            elif i == pow(2,2*N)-1 and j != 0: score = 1
            elif i != pow(2,2*N)-1 and j == 0: score = -1
            else: score = 0
            if score == 1: curr_scores[(i,j,-1)] = score #winning for first player
            elif score == -1: curr_scores[(i,j,1)] = score #winning for second player
            elif score == 0:
                curr_scores[(i,j,1)] = uniform(-1, 1)
                curr_scores[(i,j,-1)] = uniform(-1, 1)

In [18]:
#Filling out the rolls space.
rolls = []
for x in range(1, N+1):
    for y in range(1, N+1):
        rolls.append((x,y))

In [19]:
#This function covers points 3 and 4 from the algorithm presented here: 
#https://bg1992.home.blog/2020/08/31/optimal-strategy/
def f():
    diffs = 0
    for state in curr_scores:
        score = curr_scores[state]
        if score != 1 and score != -1:
            em = 0
            for roll in rolls:
                x, y = roll
                next_states = []
                if state[2] == 1: #first player to move

                    _choice = (state[0] ^ pow(2,x-1), state[1], state[2]*(-1))
                    if _choice[0] != _choice[1]: next_states.append(_choice)
                    _choice = (state[0] ^ pow(2,y-1), state[1], state[2]*(-1))
                    if _choice[0] != _choice[1]: next_states.append(_choice)
                    _choice = (state[0] ^ pow(2,x+y-1), state[1], state[2]*(-1))
                    if _choice[0] != _choice[1]: next_states.append(_choice)

                    em += max(map(lambda x: curr_scores[x], next_states)) #choosing maximum of values of the children states

                else: #second player to move

                    _choice = (state[0], state[1] ^ pow(2,x-1), state[2]*(-1))
                    if _choice[0] != _choice[1]: next_states.append(_choice)
                    _choice = (state[0], state[1] ^ pow(2,y-1), state[2]*(-1))
                    if _choice[0] != _choice[1]: next_states.append(_choice)
                    _choice = (state[0], state[1] ^ pow(2,x+y-1), state[2]*(-1))
                    if _choice[0] != _choice[1]: next_states.append(_choice)

                    em += min(map(lambda x: curr_scores[x], next_states)) #choosing minimum of values of the children states

            em /= pow(N,2)
            diffs += abs(score - em)
            curr_scores[state] = em

    return diffs

In [20]:
#Iteration process, till the tolerance is breached.
ind = 0
while 1:
    diffs = f()
    if diffs < tolerance: break
    print('Iteration no', ind, 'diffs:', diffs)
    ind += 1

Iteration no 0 diffs: 73956.10102605289
Iteration no 1 diffs: 38421.74891158869
Iteration no 2 diffs: 11038.714605739931
Iteration no 3 diffs: 8419.191015176835
Iteration no 4 diffs: 7539.248145840974
Iteration no 5 diffs: 6764.253742952484
Iteration no 6 diffs: 6208.258678955066
Iteration no 7 diffs: 5441.875406881184
Iteration no 8 diffs: 4829.538473604544
Iteration no 9 diffs: 4145.041146398239
Iteration no 10 diffs: 3607.977038517355
Iteration no 11 diffs: 3076.147922726698
Iteration no 12 diffs: 2666.0579271170964
Iteration no 13 diffs: 2262.2193516624
Iteration no 14 diffs: 1952.227645527194
Iteration no 15 diffs: 1643.639255757479
Iteration no 16 diffs: 1410.833199693396
Iteration no 17 diffs: 1176.5958927678457
Iteration no 18 diffs: 1003.6172653143184
Iteration no 19 diffs: 827.7401624074263
Iteration no 20 diffs: 700.671654670444
Iteration no 21 diffs: 570.7610520202404
Iteration no 22 diffs: 479.1668105476879
Iteration no 23 diffs: 386.33366076944867
Iteration no 24 diffs: 3

Iteration no 187 diffs: 3.745054989387486e-08
Iteration no 188 diffs: 3.2314885302364704e-08
Iteration no 189 diffs: 2.8918121814693054e-08
Iteration no 190 diffs: 2.4951800510313038e-08
Iteration no 191 diffs: 2.233035097803926e-08
Iteration no 192 diffs: 1.9266956681994463e-08
Iteration no 193 diffs: 1.724361914975242e-08
Iteration no 194 diffs: 1.487757841055204e-08
Iteration no 195 diffs: 1.3315839023433917e-08
Iteration no 196 diffs: 1.1488508799666064e-08
Iteration no 197 diffs: 1.0283061400214594e-08


Now, some analysis. Let us run packs of $10^5$ games, assuming the following variants:

1. Both players play optimally.

2. First player plays optimally, second player 8/10 times plays optimally, 2/10 times randomly (unless he sees winning position for him in the next move).

3. First player plays optimally, second player 5/10 times plays optimally, 5/10 times randomly (unless he sees winning position for him in the next move).

4. First player plays optimally, second player 2/10 times plays optimally, 8/10 times randomly (unless he sees winning position for him in the next move).

5. First player plays optimally, second player plays randomly (unless he sees winning position for him in the next move).

6. Variant 2, but switching players.

7. Variant 3, but switching players.

8. Variant 4, but switching players.

9. Variant 5, but switching players.

Let us assume that we terminate game as a draw if a number of moves reaches $500$.

In [28]:
from random import randint, random
variants = [(1,1), (1, 0.8), (1, 0.5), (1, 0.2), (1, 0), (0.8, 1), (0.5, 1), (0.2, 1), (0, 1)]
print('Scores: first player wins, second player wins, draws')
for variant in variants:
    sims_ct = pow(10,5)
    
    res = [0,0,0]
    for i in range(sims_ct):
        result = 0
        state = (0, pow(2, 2 * N) - 1, 1)
        ct = 0
        while result < 1 and result > -1:

            #print(state)
            roll = [randint(1, N), randint(1,N)]
            x, y = roll
            next_states = []

            if state[2] == 1:

                _choice = (state[0] ^ pow(2, x - 1), state[1], state[2] * (-1))
                if _choice[0] != _choice[1]: next_states.append((_choice, curr_scores[_choice]))
                _choice = (state[0] ^ pow(2, y - 1), state[1], state[2] * (-1))
                if _choice[0] != _choice[1]: next_states.append((_choice, curr_scores[_choice]))
                _choice = (state[0] ^ pow(2, x + y - 1), state[1], state[2] * (-1))
                if _choice[0] != _choice[1]: next_states.append((_choice, curr_scores[_choice]))

                next_states.sort(key= lambda x: -x[1])
                state, result = next_states[0]
                if result != 1:
                    if random() >= variant[0]:
                        state, result = next_states[randint(0,len(next_states)-1)]

            else:

                _choice = (state[0], state[1] ^ pow(2, x - 1), state[2] * (-1))
                if _choice[0] != _choice[1]: next_states.append((_choice, curr_scores[_choice]))
                _choice = (state[0], state[1] ^ pow(2, y - 1), state[2] * (-1))
                if _choice[0] != _choice[1]: next_states.append((_choice, curr_scores[_choice]))
                _choice = (state[0], state[1] ^ pow(2, x + y - 1), state[2] * (-1))
                if _choice[0] != _choice[1]: next_states.append((_choice, curr_scores[_choice]))

                next_states.sort(key=lambda x: x[1])
                state, result = next_states[0]
                
                if result != -1:
                    if random() >= variant[1]:
                        state, result = next_states[randint(0,len(next_states)-1)]

            ct += 1
            if ct == 500: break

        if result == 1: res[0] += 1
        elif result == -1: res[1] += 1
        else: res[2] += 1

    print('Variant: ', variant, res)

Scores: first player wins, second player wins, draws
Variant:  (1, 1) [53465, 46535, 0]
Variant:  (1, 0.8) [62059, 37941, 0]
Variant:  (1, 0.5) [76773, 23227, 0]
Variant:  (1, 0.2) [90401, 9599, 0]
Variant:  (1, 0) [96425, 3575, 0]
Variant:  (0.8, 1) [43430, 56570, 0]
Variant:  (0.5, 1) [27151, 72849, 0]
Variant:  (0.2, 1) [11134, 88866, 0]
Variant:  (0, 1) [4049, 95951, 0]


As we can observe, even with some luck included, skill seems to be a crucial factor of the outcome in the long run. It is worth noticing that the first player seems to have a small advantage resulting from the fact his move is first.